In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sms
from statsmodels.iolib.summary2 import summary_col
plt.style.use('seaborn-whitegrid')

pd.options.mode.chained_assignment = None

In [2]:
pns1 = pd.read_csv('dados/pns.csv') # pns 2013
pns2 = pns1.query('idade > 17') # incluir apenas quem já poderia ter concluído o ensino médio
pns3 = pns2.query('freq_esc == 2') # apenas quem não frequenta mais a escola
pns4 = pns3.query('escolaridade == [3, 5]') # apenas os dois níveis de escolaridade
linha_vazia = pd.DataFrame([np.nan]).T
pns5 = pns4.query('rendimento > 0') # rendimento maior do que 0

print(' início: ', pns1.shape[0], '\n',
      'idade > 17: ', pns2.shape[0], '\n',
      'não frequenta a escola: ', pns3.shape[0], '\n',
      'fundamental ou médio: ', pns4.shape[0], '\n',
      'rendimento > 0: ', pns5.shape[0])

 início:  205546 
 idade > 17:  145580 
 não frequenta a escola:  129926 
 fundamental ou médio:  51155 
 rendimento > 0:  34479


In [4]:
pns = pns4

In [ ]:
pns['saude'] = (pns.percepcao <= 2).astype(int) # 1 a 2 = bom, 3 a 5 = ruim 
pns.doenca_diag = pns.doenca_diag.astype(int).replace(2, 0) # 0 = não, 1 = sim
pns.doenca_hipertensao = pns.doenca_hipertensao.replace([3, 2], [0, np.nan]) # 0 = não, 1 = sim
pns.plano = pns.plano.astype(int).replace(2, 0) # 0 = não, 1 = sim
pns.sexo = pns.sexo.astype(int).replace(2, 0) # 0 = feminino, 1 = masculino (REVER)
pns.fuma = pns.fuma.replace([3, 2], [0, np.nan]) # 0 = não, 1 = sim 
pns.exercicio_3meses = pns.exercicio_3meses.replace(2, 0) # 0 = não, 1 = sim
pns.escolaridade = pns.escolaridade.replace([3,5], [0,1]) # 0 = fundamental, 1 = médio

# renomeei apenas aqui. É melhor renomear no arquivo original?
pns['fumante'] = pns.fuma
pns['consulta'] = pns.quantidade_consulta_medico
pns['exercicio'] = pns.exercicio_3meses

# podemos dividir o rendimento por 100 para melhorar a interpretação?
pns['rendimento'] = pns.rendimento/100 

### **Modelo 1**

In [6]:
modelo1 = smf.logit('saude ~ escolaridade + sexo + idade', data=pns).fit()
modelo1.summary()

Optimization terminated successfully.
         Current function value: 0.538338
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  saude   No. Observations:                51155
Model:                          Logit   Df Residuals:                    51151
Method:                           MLE   Df Model:                            3
Date:                Mon, 18 Mar 2019   Pseudo R-squ.:                 0.04849
Time:                        14:50:20   Log-Likelihood:                -27539.
converged:                       True   LL-Null:                       -28942.
                                        LLR p-value:                     0.000
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        1.8379      0.038     48.640      0.000       1.764       1.912
escolaridade     0.4363      0.023     18.803      0.000       0.391       0.482
sexo             0.3351      0.021     15.814      0.000       0.294       0.377
idade           -0.0308      0.001    -43.566      0.000      -0.032      -0.029
================================================================================
"""

In [7]:
np.exp(modelo1.params)

Intercept       6.283112
escolaridade    1.546914
sexo            1.398133
idade           0.969634
dtype: float64

- possuir ensino médio aumenta em 55% ((1,55 - 1)) * 100) a chance de o indivíduo se considerar saudável.
- a chance de os homens se considerarem saudáveis é 39% maior em comparação às mulheres.
- um ano a mais de idade diminui em 4% ((0.96 - 1) * 100) a chance de o indivíduo se considerar saudável.

### **Modelo 2**

In [10]:
modelo2 = smf.logit('saude ~ escolaridade + sexo + idade + rendimento', data=pns).fit()
modelo2.summary()

Optimization terminated successfully.
         Current function value: 0.501748
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  saude   No. Observations:                34479
Model:                          Logit   Df Residuals:                    34474
Method:                           MLE   Df Model:                            4
Date:                Mon, 18 Mar 2019   Pseudo R-squ.:                 0.04405
Time:                        14:50:49   Log-Likelihood:                -17300.
converged:                       True   LL-Null:                       -18097.
                                        LLR p-value:                     0.000
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        1.9869      0.054     36.548      0.000       1.880       2.093
escolaridade     0.3904      0.030     12.850      0.000       0.331       0.450
sexo             0.2575      0.028      9.320      0.000       0.203       0.312
idade           -0.0367      0.001    -32.003      0.000      -0.039      -0.034
rendimento       0.0193      0.001     13.725      0.000       0.017       0.022
================================================================================
"""

In [11]:
np.exp(modelo2.params)

Intercept       7.292530
escolaridade    1.477557
sexo            1.293717
idade           0.963980
rendimento      1.019476
dtype: float64

- possuir ensino médio aumenta em 47% a chance de o indivíduo se considerar saudável
- a chance de os homens se considerarem saudávies é 29% maior em comparação às mulheres.
- cada ano a mais de idade diminui em 4% a chance de ser considerar saudável.
- rendimento: cada 100,00 a mais no rendimento aumenta em 1,93% a chande de ao indivíduo se considerar saudável.

### **Modelo 3**

In [12]:
modelo3 = smf.logit('saude ~ escolaridade + sexo + idade + rendimento + plano + fumante + consulta + exercicio',
          data=pns).fit()
modelo3.summary()

Optimization terminated successfully.
         Current function value: 0.521064
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  saude   No. Observations:                 9437
Model:                          Logit   Df Residuals:                     9428
Method:                           MLE   Df Model:                            8
Date:                Mon, 18 Mar 2019   Pseudo R-squ.:                 0.09044
Time:                        14:50:55   Log-Likelihood:                -4917.3
converged:                       True   LL-Null:                       -5406.2
                                        LLR p-value:                8.660e-206
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        2.0641      0.105     19.700      0.000       1.859       2.269
escolaridade     0.3318      0.058      5.758      0.000       0.219       0.445
sexo             0.1567      0.054      2.917      0.004       0.051       0.262
idade           -0.0350      0.002    -16.318      0.000      -0.039      -0.031
rendimento       0.0120      0.002      5.007      0.000       0.007       0.017
plano            0.7335      0.058     12.579      0.000       0.619       0.848
fumante         -0.1301      0.081     -1.608      0.108      -0.289       0.028
consulta        -0.1189      0.007    -17.333      0.000      -0.132      -0.105
exercicio        0.2625      0.055      4.785      0.000       0.155       0.370
================================================================================
"""

In [13]:
np.exp(modelo3.params)

Intercept       7.877870
escolaridade    1.393495
sexo            1.169688
idade           0.965561
rendimento      1.012073
plano           2.082338
fumante         0.878035
consulta        0.887873
exercicio       1.300166
dtype: float64

- possuir o ensino médio aumenta em 39% a chance de o indivíduo se considerar saudável.
- a chance de os homens se considerarem saudáveis é 17% maior em comparação às mulheres.
- um ano a mais de idade diminui em 4% a chance de o indivíduo se considerar saudável.
- cada 100 reais a mais aumenta em 1,21% a chance de se considerar saudável.
- possuir um plano de saúde aumenta em 108% a chance de o indivíduo se considerar saudável.
- uma consulta a mais no médico nos últimos 12 meses diminui em 11% a chance de o indivíduo se considerar saudável.
- ter praticado algum exercício físico nos últimos 3 meses aumenta em 30% a chance de o indivíduo de considerar saudável.

### **Tabela**

In [12]:
modelo1 = smf.logit('saude ~ escolaridade + sexo + idade', data=pns).fit()
modelo2 = smf.logit('saude ~ escolaridade + sexo + idade + rendimento', data=pns).fit()
modelo3 = smf.logit('saude ~ escolaridade + sexo + idade + rendimento + plano + fumante + consulta+ exercicio',
          data=pns).fit()

info_dict = {'Pseudo R-quadrado' : lambda x: "{:.2f}".format(x.prsquared),
             'Nº de observações' : lambda x: "{0:d}".format(int(x.nobs))}

tabela = summary_col(results = [modelo1, modelo2, modelo3],
                            float_format='%0.2f',
                            model_names=['Modelo 1', 'Modelo 2', 'Modelo 3'],
                            info_dict=info_dict,
                            regressor_order=['Intercept', 'escolaridade', 'sexo', 'idade',
                                             'rendimento', 'plano', 'fumante', 'consulta',
                                            'exercicio'])

tabela.add_title('Resultados das regressões')

print(tabela)

Optimization terminated successfully.
         Current function value: 0.538338
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.501748
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.521064
         Iterations 6
         Resultados das regressões
                  Modelo 1 Modelo 2 Modelo 3
--------------------------------------------
Intercept         1.84     1.99     2.06    
                  (0.04)   (0.05)   (0.10)  
escolaridade      0.44     0.39     0.33    
                  (0.02)   (0.03)   (0.06)  
mulher            0.34     0.26     0.16    
                  (0.02)   (0.03)   (0.05)  
idade             -0.03    -0.04    -0.04   
                  (0.00)   (0.00)   (0.00)  
rendimento                 0.02     0.01    
                           (0.00)   (0.00)  
plano                               0.73    
                                    (0.06)  
fumante                      

In [13]:
b = tabela.tables[0]
expmodelo1 = np.exp(modelo1.params)
expmodelo2 = np.exp(modelo2.params)
expmodelo3 = np.exp(modelo3.params)
b['exp1'] = expmodelo1
b['exp2'] = expmodelo2
b['exp3'] = expmodelo3
b = b.loc[:, ['Modelo 1', 'exp1', 'Modelo 2', 'exp2', 'Modelo 3', 'exp3']]
b = b.replace(np.nan, '')
b.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Modelo 1</th>\n      <th>exp1</th>\n      <th>Modelo 2</th>\n      <th>exp2</th>\n      <th>Modelo 3</th>\n      <th>exp3</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Intercept</th>\n      <td>1.84</td>\n      <td>6.28311</td>\n      <td>1.99</td>\n      <td>7.29253</td>\n      <td>2.06</td>\n      <td>7.87787</td>\n    </tr>\n    <tr>\n      <th></th>\n      <td>(0.04)</td>\n      <td></td>\n      <td>(0.05)</td>\n      <td></td>\n      <td>(0.10)</td>\n      <td></td>\n    </tr>\n    <tr>\n      <th>escolaridade</th>\n      <td>0.44</td>\n      <td>1.54691</td>\n      <td>0.39</td>\n      <td>1.47756</td>\n      <td>0.33</td>\n      <td>1.3935</td>\n    </tr>\n    <tr>\n      <th></th>\n      <td>(0.02)</td>\n      <td></td>\n      <td>(0.03)</td>\n      <td></td>\n      <td>(0.06)</td>\n      <td></td>\n    </tr>\n    <tr>\n      <th>mulher</th>\n      <td

### **Modelos com outras variáveis resposta**

### **Modelo 4**

In [14]:
modelo4 = smf.logit('doenca_diag ~ escolaridade + sexo + idade + rendimento + plano + fumante + quantidade_consulta_medico + exercicio_3meses', data=pns).fit()
modelo4.summary()

Optimization terminated successfully.
         Current function value: 0.377381
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            doenca_diag   No. Observations:                 9437
Model:                          Logit   Df Residuals:                     9428
Method:                           MLE   Df Model:                            8
Date:                Mon, 18 Mar 2019   Pseudo R-squ.:                 0.09954
Time:                        14:51:39   Log-Likelihood:                -3561.3
converged:                       True   LL-Null:                       -3955.0
                                        LLR p-value:                1.075e-164
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                     -4.2622      0.138    -30.838      0.000      -4.533      -3.991
escolaridade                  -0.0576      0.072     -0.797      0.426      -0.199       0.084
sexo                          -0.2466      0.065     -3.796      0.000      -0.374      -0.119
idade                          0.0566      0.003     21.774      0.000       0.051       0.062
rendimento                    -0.0012      0.002     -0.759      0.448      -0.004       0.002
plano                          0.0168      0.066      0.257      0.797      -0.112       0.145
fumante                        0.0899      0.098      0.921      0.357      -0.101       0.281
quantidade_consulta_medico     0.0940      0.007     13.666      0.000       0.081       0.107
exercicio_3meses              -0.0261      0.066     -0.396      0.692      -0.156       0.103
==============================================================================================
"""

In [15]:
np.exp(modelo4.params)

Intercept                     0.014091
escolaridade                  0.944038
sexo                          0.781452
idade                         1.058186
rendimento                    0.998755
plano                         1.016982
fumante                       1.094105
quantidade_consulta_medico    1.098563
exercicio_3meses              0.974200
dtype: float64

- homens têm uma chance 22% menor de serem diagnosticados com alguma doença crônica.
- um ano a mais de idade aumenta em 6% a chance de ser diagnosticado com alguma doença crônica.
- uma cnosulta a mais no médico no último ano aumenta em 9,86% a chance de o indivíduo ser diagnosticado com alguma doença crônica.
- variáveis escolaridade, plano, rendimento, fuma e exercicios_3meses não foram significativas.

### **Modelo 5**

In [16]:
modelo5 = smf.logit('doenca_hipertensao ~ escolaridade + sexo + idade + rendimento + plano + fumante + consulta + exercicio',
          data=pns).fit()
modelo5.summary()

Optimization terminated successfully.
         Current function value: 0.389674
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:     doenca_hipertensao   No. Observations:                 9136
Model:                          Logit   Df Residuals:                     9127
Method:                           MLE   Df Model:                            8
Date:                Mon, 18 Mar 2019   Pseudo R-squ.:                  0.1393
Time:                        14:52:23   Log-Likelihood:                -3560.1
converged:                       True   LL-Null:                       -4136.1
                                        LLR p-value:                2.127e-243
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -4.8371      0.142    -34.002      0.000      -5.116      -4.558
escolaridade    -0.1259      0.070     -1.790      0.073      -0.264       0.012
sexo            -0.0353      0.064     -0.555      0.579      -0.160       0.089
idade            0.0787      0.003     29.169      0.000       0.073       0.084
rendimento      -0.0029      0.002     -1.699      0.089      -0.006       0.000
plano           -0.1396      0.065     -2.132      0.033      -0.268      -0.011
fumante         -0.0980      0.098     -1.001      0.317      -0.290       0.094
consulta         0.0606      0.007      8.795      0.000       0.047       0.074
exercicio       -0.1492      0.066     -2.268      0.023      -0.278      -0.020
================================================================================
"""

In [17]:
np.exp(modelo5.params)

Intercept       0.007930
escolaridade    0.881737
sexo            0.965351
idade           1.081903
rendimento      0.997116
plano           0.869676
fumante         0.906619
consulta        1.062449
exercicio       0.861431
dtype: float64

- a variável idade mostra que um ano a mais de idade aumenta em 8% a chance de o indivíduo ser diagnosticado com hipertensão.
- possuir plano de saúde diminui em 13,03% a chance de o indivíduo ser diagnosticado com hipertensão.
- a variável que mostra a quantidade de vezes que a pessoa foi ao médico no último ano mostra que uma ida a mais aumenta em 6% a chance de a pessoa ser diagnosticada com hipertensão.
- por fim, ter feito algum exercício físico nos últimos três meses diminui em 13,86% a chance de o indivíduo ser diagnosticado com hipertensão.
- escolaridade, sexo, rendimento e fumante não foram significativas.

### **Tabela com os resultados dos dois modelos**

In [19]:
modelo4 = smf.logit('doenca_diag ~ escolaridade + mulher + idade + rendimento + plano + fumante + consulta + exercicio',
                    data=pns).fit()
modelo5 = smf.logit('doenca_hipertensao ~ escolaridade + mulher + idade + rendimento + plano + fumante + consulta + exercicio',
          data=pns).fit()

info_dict={'Pseudo R-quadrado' : lambda x: "{:.2f}".format(x.prsquared),
           'Nº de observações' : lambda x: "{0:d}".format(int(x.nobs))}

tabela2 = summary_col(results=[modelo4, modelo5],
                            float_format='%0.2f',
                            model_names=['Modelo 4', 'Modelo 5'],
                            info_dict=info_dict,
                            regressor_order=['Intercept', 'escolaridade', 'mulher', 'idade',
                                             'rendimento', 'plano', 'fumante', 'consulta',
                                            'exercicio'])

tabela.add_title('Resultados das regressões')

print(tabela2)

Optimization terminated successfully.
         Current function value: 0.377381
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.389674
         Iterations 7

                  Modelo 4 Modelo 5
-----------------------------------
Intercept         -4.26    -4.84   
                  (0.14)   (0.14)  
escolaridade      -0.06    -0.13   
                  (0.07)   (0.07)  
mulher            -0.25    -0.04   
                  (0.06)   (0.06)  
idade             0.06     0.08    
                  (0.00)   (0.00)  
rendimento        -0.00    -0.00   
                  (0.00)   (0.00)  
plano             0.02     -0.14   
                  (0.07)   (0.07)  
fumante           0.09     -0.10   
                  (0.10)   (0.10)  
consulta          0.09     0.06    
                  (0.01)   (0.01)  
exercicio         -0.03    -0.15   
                  (0.07)   (0.07)  
Pseudo R-quadrado 0.10     0.14    
Nº de observações 9437     9136    
Sta

In [20]:
c = tabela2.tables[0]
expmodelo4 = np.exp(modelo4.params)
expmodelo5 = np.exp(modelo5.params)
c['exp4'] = expmodelo4
c['exp5'] = expmodelo5
c = c.loc[:, ['Modelo 4', 'exp4', 'Modelo 5', 'exp5']]
c = c.replace(np.nan, '')
c.to_html()
# copiar e colar especial

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Modelo 4</th>\n      <th>exp4</th>\n      <th>Modelo 5</th>\n      <th>exp5</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Intercept</th>\n      <td>-4.26</td>\n      <td>0.0140914</td>\n      <td>-4.84</td>\n      <td>0.00793032</td>\n    </tr>\n    <tr>\n      <th></th>\n      <td>(0.14)</td>\n      <td></td>\n      <td>(0.14)</td>\n      <td></td>\n    </tr>\n    <tr>\n      <th>escolaridade</th>\n      <td>-0.06</td>\n      <td>0.944038</td>\n      <td>-0.13</td>\n      <td>0.881737</td>\n    </tr>\n    <tr>\n      <th></th>\n      <td>(0.07)</td>\n      <td></td>\n      <td>(0.07)</td>\n      <td></td>\n    </tr>\n    <tr>\n      <th>mulher</th>\n      <td>-0.25</td>\n      <td>0.781452</td>\n      <td>-0.04</td>\n      <td>0.965351</td>\n    </tr>\n    <tr>\n      <th></th>\n      <td>(0.06)</td>\n      <td></td>\n      <td>(0.06)</td>\n      <td></td>\n 